In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, json
import glob
import csv
from pandas import DataFrame
import fasttext
import csv

In [27]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

--2021-03-29 12:54:18--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz.2’

lid.176.ftz.2       100%[===================>] 916.03K  2.28MB/s    in 0.4s    

2021-03-29 12:54:19 (2.28 MB/s) - ‘lid.176.ftz.2’ saved [938013/938013]



In [28]:
# replacing `'./archive/metadata.csv ` with the path to where you saved the `archive/metadata.csv` directory that you downloaded; 
original_data_path = './archive/metadata.csv'
df = pd.read_csv(original_data_path)

FileNotFoundError: [Errno 2] No such file or directory: './archive/metadata.csv'

In [111]:
#drop rows that have empty pdf_json cels
df.dropna(subset=['pdf_json_files'], inplace=True)
df.dropna(subset=['abstract'], inplace=True)

In [113]:
df["pdf_json_files"] = df["pdf_json_files"].str.split(";").str[0]

In [115]:
#creating our functions
#function to format body_text into block of text
def format_body(body_text):
    body = ""
    for sec in body_text:
        body = body + sec['text'] + " "
    return body

In [119]:
path_to_json ='./archive/'

In [120]:
body_text = []
for filename in df['pdf_json_files']:
    filename = path_to_json + filename 
    my_json_file = json.load(open(filename, 'r'))
    body_text.append(format_body(my_json_file['body_text']))
   

In [121]:
covid_df = pd.DataFrame(columns=['title','abstract','body_text'])
covid_df['title'] = df['title']
covid_df['abstract'] = df['abstract']
covid_df['body_text'] = body_text
covid_df.dropna(subset=['body_text'], inplace=True)

In [122]:
model = fasttext.load_model('lid.176.ftz')

In [123]:
def check_english(text):
    if 'en' in str(model.predict(text)[0][0]):
        return True
    else:
        return False

In [124]:
covid_df['is_english'] = covid_df.apply(lambda x: check_english(x['body_text']),axis=1)

In [125]:
#check english
covid_english_df = covid_df[covid_df['is_english'] == True] 

In [126]:
covid_english_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136809 entries, 0 to 490901
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       136809 non-null  object
 1   abstract    136809 non-null  object
 2   body_text   136809 non-null  object
 3   is_english  136809 non-null  bool  
dtypes: bool(1), object(3)
memory usage: 4.3+ MB


In [134]:
covid_english_df.to_csv('covid_english.csv')

In [3]:
df = pd.read_csv('covid_english.csv')

In [4]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.8*len(df)), int(.9*len(df))])

In [40]:
train.reset_index(drop=True,inplace=True)
validate.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)


In [41]:
train_source = open('train.source','w')
train_target = open('train.target','w')
for i, row in train.iterrows():
    train_source.write("%s\n" % row['body_text'])
    train_target.write("%s\n" % row['abstract'])
train_source.close()
train_target.close()

In [ ]:

val_source = open('val.source','w')
val_target = open('val.target','w')
for i, row in validate.iterrows():
    val_source.write("%s\n" % row['body_text'])
    val_target.write("%s\n" % row['abstract'])
val_source.close()
val_target.close()

In [ ]:
test_source = open('test.source','w')
test_target = open('test.target','w')
for i, row in test.iterrows():
    # if i >= 13679:
    #     print(row)
    test_source.write("%s\n" % row['body_text'])
    test_target.write("%s\n" % row['abstract'])
test_source.close()
test_target.close()